# Exercices : Bibliotheque Standard et Fichiers

⚡ **Intermediaire** | ⏱ **2h** | 📚 **Sections validees : 04-Bibliotheque-Standard**

## Objectifs
- Maitriser les expressions regulieres (regex)
- Manipuler fichiers et dossiers avec pathlib
- Creer des scripts CLI avec argparse
- Utiliser dataclasses et Enum
- Gerer la configuration avec dotenv

## Sections couvertes
1. Regex (2 exercices)
2. Fichiers et pathlib (2 exercices)
3. CLI avec argparse (1 exercice)
4. Dataclasses et Enum (2 exercices)
5. Configuration avec dotenv (1 exercice)

---
## Section 1 : Expressions Regulieres

### Exercice 1.1 - Parser des Logs (FACILE)

Creez une fonction `parser_logs(log_text)` qui extrait des logs au format :
```
2024-01-15 14:30:22 [ERROR] Database connection failed
2024-01-15 14:30:25 [INFO] Retrying connection...
2024-01-15 14:30:28 [WARNING] Slow query detected (3.5s)
```

La fonction doit retourner une liste de dictionnaires avec :
- `date` : str
- `heure` : str
- `niveau` : str (ERROR, INFO, WARNING)
- `message` : str

In [ ]:
# Votre code ici


### Solution 1.1

In [ ]:
# Solution
import re

def parser_logs(log_text):
    """
    Parse un texte de logs et extrait les informations structurees
    """
    # Pattern regex : date heure [niveau] message
    pattern = r'(\d{4}-\d{2}-\d{2})\s+(\d{2}:\d{2}:\d{2})\s+\[(\w+)\]\s+(.+)'
    
    resultats = []
    
    for ligne in log_text.strip().split('\n'):
        match = re.match(pattern, ligne)
        if match:
            date, heure, niveau, message = match.groups()
            resultats.append({
                'date': date,
                'heure': heure,
                'niveau': niveau,
                'message': message
            })
    
    return resultats


# Test
logs = """
2024-01-15 14:30:22 [ERROR] Database connection failed
2024-01-15 14:30:25 [INFO] Retrying connection...
2024-01-15 14:30:28 [WARNING] Slow query detected (3.5s)
2024-01-15 14:30:30 [INFO] Connection established
2024-01-15 14:31:00 [ERROR] Timeout on request /api/users
"""

parsed = parser_logs(logs)
print(f"Nombre de logs : {len(parsed)}\n")

for log in parsed:
    print(f"[{log['date']} {log['heure']}] {log['niveau']:8s} : {log['message']}")

# Filtrer par niveau
print("\nErreurs uniquement :")
erreurs = [log for log in parsed if log['niveau'] == 'ERROR']
for err in erreurs:
    print(f"  - {err['message']}")

### Exercice 1.2 - Validation de Donnees (MOYEN)

Creez des fonctions de validation avec regex :

1. `valider_email(email)` : RFC 5322 simplifie
2. `valider_telephone_fr(tel)` : formats 06/07, +33, espaces optionnels
3. `valider_iban_fr(iban)` : IBAN francais (FR + 2 chiffres + 23 alphanumeriques)
4. `extraire_urls(texte)` : extrait toutes les URLs d'un texte

Testez avec des exemples valides et invalides.

In [ ]:
# Votre code ici


### Solution 1.2

In [ ]:
# Solution
import re

def valider_email(email):
    """
    Valide un email (version simplifiee RFC 5322)
    """
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return bool(re.match(pattern, email))


def valider_telephone_fr(tel):
    """
    Valide un numero de telephone francais
    Formats acceptes :
    - 06 12 34 56 78
    - 0612345678
    - +33 6 12 34 56 78
    - +33612345678
    """
    # Nettoyer les espaces
    tel_clean = tel.replace(' ', '').replace('.', '').replace('-', '')
    
    # Format francais
    pattern_fr = r'^0[67]\d{8}$'
    # Format international
    pattern_intl = r'^\+33[67]\d{8}$'
    
    return bool(re.match(pattern_fr, tel_clean) or re.match(pattern_intl, tel_clean))


def valider_iban_fr(iban):
    """
    Valide un IBAN francais
    Format : FR76 1234 5678 9012 3456 7890 123
    """
    # Nettoyer les espaces
    iban_clean = iban.replace(' ', '').upper()
    
    # Pattern : FR + 2 chiffres + 23 caracteres alphanumeriques
    pattern = r'^FR\d{2}[A-Z0-9]{23}$'
    
    return bool(re.match(pattern, iban_clean))


def extraire_urls(texte):
    """
    Extrait toutes les URLs d'un texte
    """
    pattern = r'https?://[^\s<>"]+|www\.[^\s<>"]+'
    return re.findall(pattern, texte)


# Tests
print("=== VALIDATION EMAIL ===")
emails_test = [
    "john.doe@example.com",
    "invalid.email@",
    "test@domain",
    "user+tag@company.co.uk"
]
for email in emails_test:
    valide = valider_email(email)
    print(f"{email:30s} : {'VALIDE' if valide else 'INVALIDE'}")

print("\n=== VALIDATION TELEPHONE ===")
telephones_test = [
    "06 12 34 56 78",
    "0612345678",
    "+33 6 12 34 56 78",
    "+33612345678",
    "05 12 34 56 78",  # Invalide (fixe)
    "06123456789"  # Invalide (trop long)
]
for tel in telephones_test:
    valide = valider_telephone_fr(tel)
    print(f"{tel:25s} : {'VALIDE' if valide else 'INVALIDE'}")

print("\n=== VALIDATION IBAN ===")
ibans_test = [
    "FR76 1234 5678 9012 3456 7890 123",
    "FR7612345678901234567890123",
    "DE89370400440532013000",  # Allemand
    "FR76 1234 5678"  # Trop court
]
for iban in ibans_test:
    valide = valider_iban_fr(iban)
    print(f"{iban:40s} : {'VALIDE' if valide else 'INVALIDE'}")

print("\n=== EXTRACTION URLs ===")
texte = """
Visitez notre site https://www.example.com pour plus d'infos.
Documentation : https://docs.python.org/3/library/re.html
Ou consultez www.github.com/username/repo
"""
urls = extraire_urls(texte)
print(f"URLs trouvees : {len(urls)}")
for url in urls:
    print(f"  - {url}")

---
## Section 2 : Fichiers et Pathlib

### Exercice 2.1 - Analyse de Structure de Dossier (FACILE)

Creez une fonction `analyser_dossier(path)` qui :
- Compte le nombre de fichiers et sous-dossiers
- Calcule la taille totale
- Liste les extensions presentes avec leur frequence
- Retourne un dictionnaire avec ces informations

Utilisez `pathlib.Path` et ses methodes.

In [ ]:
# Votre code ici


### Solution 2.1

In [ ]:
# Solution
from pathlib import Path
from collections import Counter

def analyser_dossier(path):
    """
    Analyse la structure d'un dossier
    """
    chemin = Path(path)
    
    if not chemin.exists():
        raise FileNotFoundError(f"Le chemin {path} n'existe pas")
    
    if not chemin.is_dir():
        raise NotADirectoryError(f"{path} n'est pas un dossier")
    
    # Compteurs
    nb_fichiers = 0
    nb_dossiers = 0
    taille_totale = 0
    extensions = []
    
    # Parcourir recursivement
    for item in chemin.rglob('*'):
        if item.is_file():
            nb_fichiers += 1
            taille_totale += item.stat().st_size
            if item.suffix:
                extensions.append(item.suffix.lower())
        elif item.is_dir():
            nb_dossiers += 1
    
    # Compter les extensions
    compteur_ext = Counter(extensions)
    
    return {
        'chemin': str(chemin.absolute()),
        'nb_fichiers': nb_fichiers,
        'nb_dossiers': nb_dossiers,
        'taille_totale': taille_totale,
        'taille_lisible': formater_taille(taille_totale),
        'extensions': dict(compteur_ext.most_common())
    }


def formater_taille(taille_bytes):
    """
    Formate une taille en bytes de maniere lisible
    """
    for unite in ['B', 'KB', 'MB', 'GB', 'TB']:
        if taille_bytes < 1024.0:
            return f"{taille_bytes:.2f} {unite}"
        taille_bytes /= 1024.0
    return f"{taille_bytes:.2f} PB"


# Test (sur le dossier courant)
try:
    analyse = analyser_dossier('.')
    
    print("=== ANALYSE DU DOSSIER ===")
    print(f"Chemin : {analyse['chemin']}")
    print(f"Fichiers : {analyse['nb_fichiers']}")
    print(f"Dossiers : {analyse['nb_dossiers']}")
    print(f"Taille totale : {analyse['taille_lisible']}")
    print("\nExtensions :")
    for ext, count in analyse['extensions'].items():
        print(f"  {ext:10s} : {count:3d} fichier(s)")
except Exception as e:
    print(f"Erreur : {e}")

### Exercice 2.2 - Organisateur de Fichiers (MOYEN)

Creez un script `organiser_fichiers(source_dir, mode='copier')` qui :
- Trie les fichiers par extension dans des sous-dossiers
- Categories : images, documents, videos, audio, archives, autres
- Mode 'copier' : copie les fichiers
- Mode 'deplacer' : deplace les fichiers
- Cree un rapport des operations

Mapping d'extensions :
```python
CATEGORIES = {
    'images': ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg'],
    'documents': ['.pdf', '.doc', '.docx', '.txt', '.xlsx', '.pptx'],
    'videos': ['.mp4', '.avi', '.mkv', '.mov'],
    'audio': ['.mp3', '.wav', '.flac', '.aac'],
    'archives': ['.zip', '.tar', '.gz', '.rar', '.7z']
}
```

In [ ]:
# Votre code ici


### Solution 2.2

In [ ]:
# Solution
from pathlib import Path
import shutil
from collections import defaultdict

CATEGORIES = {
    'images': ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg', '.webp'],
    'documents': ['.pdf', '.doc', '.docx', '.txt', '.xlsx', '.pptx', '.odt', '.csv'],
    'videos': ['.mp4', '.avi', '.mkv', '.mov', '.wmv', '.flv'],
    'audio': ['.mp3', '.wav', '.flac', '.aac', '.ogg', '.m4a'],
    'archives': ['.zip', '.tar', '.gz', '.rar', '.7z', '.bz2'],
    'code': ['.py', '.js', '.java', '.cpp', '.c', '.html', '.css', '.sql']
}

def obtenir_categorie(extension):
    """
    Determine la categorie d'un fichier selon son extension
    """
    extension = extension.lower()
    for categorie, extensions in CATEGORIES.items():
        if extension in extensions:
            return categorie
    return 'autres'


def organiser_fichiers(source_dir, mode='copier', dry_run=True):
    """
    Organise les fichiers d'un dossier par categorie
    
    Args:
        source_dir: Dossier source
        mode: 'copier' ou 'deplacer'
        dry_run: Si True, simule sans executer
    """
    source = Path(source_dir)
    
    if not source.exists() or not source.is_dir():
        raise ValueError(f"{source_dir} n'est pas un dossier valide")
    
    if mode not in ['copier', 'deplacer']:
        raise ValueError("Mode doit etre 'copier' ou 'deplacer'")
    
    # Rapport
    rapport = defaultdict(list)
    erreurs = []
    
    # Parcourir les fichiers (non recursif)
    for fichier in source.iterdir():
        if not fichier.is_file():
            continue
        
        # Determiner la categorie
        categorie = obtenir_categorie(fichier.suffix)
        
        # Creer le dossier de destination
        dest_dir = source / categorie
        dest_file = dest_dir / fichier.name
        
        # Enregistrer dans le rapport
        rapport[categorie].append(fichier.name)
        
        if dry_run:
            print(f"[DRY-RUN] {mode:8s} {fichier.name} -> {categorie}/")
            continue
        
        try:
            # Creer le dossier si necessaire
            dest_dir.mkdir(exist_ok=True)
            
            # Gerer les doublons
            if dest_file.exists():
                stem = dest_file.stem
                suffix = dest_file.suffix
                counter = 1
                while dest_file.exists():
                    dest_file = dest_dir / f"{stem}_{counter}{suffix}"
                    counter += 1
            
            # Copier ou deplacer
            if mode == 'copier':
                shutil.copy2(fichier, dest_file)
            else:  # deplacer
                shutil.move(str(fichier), str(dest_file))
            
            print(f"[OK] {mode:8s} {fichier.name} -> {categorie}/")
        
        except Exception as e:
            erreur = f"Erreur avec {fichier.name} : {e}"
            erreurs.append(erreur)
            print(f"[ERREUR] {erreur}")
    
    # Afficher le rapport
    print("\n" + "="*50)
    print("RAPPORT D'ORGANISATION")
    print("="*50)
    
    for categorie, fichiers in sorted(rapport.items()):
        print(f"\n{categorie.upper()} ({len(fichiers)} fichier(s)) :")
        for f in fichiers[:5]:  # Afficher max 5
            print(f"  - {f}")
        if len(fichiers) > 5:
            print(f"  ... et {len(fichiers) - 5} autre(s)")
    
    if erreurs:
        print(f"\n{len(erreurs)} erreur(s) :")
        for err in erreurs:
            print(f"  - {err}")
    
    return rapport, erreurs


# Test en mode dry-run
print("Test en mode simulation (dry-run) sur le dossier courant :\n")
try:
    rapport, erreurs = organiser_fichiers('.', mode='copier', dry_run=True)
except Exception as e:
    print(f"Erreur : {e}")

---
## Section 3 : CLI avec Argparse

### Exercice 3.1 - Mini-Grep (DIFFICILE)

Creez un script CLI `mini_grep.py` qui simule grep :

**Arguments :**
- `pattern` : motif a rechercher (requis)
- `files` : fichiers a analyser (requis, multiple)
- `--ignore-case / -i` : insensible a la casse
- `--line-number / -n` : afficher les numeros de ligne
- `--count / -c` : afficher seulement le nombre de matches
- `--regex / -r` : traiter le pattern comme regex

Implementez la fonction et testez-la.

In [ ]:
# Votre code ici


### Solution 3.1

In [ ]:
# Solution
import argparse
import re
from pathlib import Path

def mini_grep(pattern, files, ignore_case=False, line_number=False, count=False, use_regex=False):
    """
    Fonction principale du mini-grep
    """
    # Preparer le pattern
    if use_regex:
        flags = re.IGNORECASE if ignore_case else 0
        try:
            regex = re.compile(pattern, flags)
        except re.error as e:
            print(f"Erreur regex : {e}")
            return
    else:
        # Recherche simple
        search_pattern = pattern.lower() if ignore_case else pattern
    
    # Parcourir les fichiers
    for filepath in files:
        path = Path(filepath)
        
        if not path.exists():
            print(f"Fichier introuvable : {filepath}")
            continue
        
        if not path.is_file():
            print(f"Pas un fichier : {filepath}")
            continue
        
        try:
            with open(path, 'r', encoding='utf-8') as f:
                lignes = f.readlines()
            
            matches = []
            
            for num_ligne, ligne in enumerate(lignes, 1):
                # Test de match
                if use_regex:
                    match = regex.search(ligne)
                else:
                    ligne_test = ligne.lower() if ignore_case else ligne
                    match = search_pattern in ligne_test
                
                if match:
                    matches.append((num_ligne, ligne.rstrip()))
            
            # Afficher les resultats
            if count:
                print(f"{filepath}: {len(matches)}")
            elif matches:
                if len(files) > 1:
                    print(f"\n{filepath}:")
                for num, ligne in matches:
                    if line_number:
                        print(f"{num}:{ligne}")
                    else:
                        print(ligne)
        
        except UnicodeDecodeError:
            print(f"Erreur d'encodage : {filepath}")
        except Exception as e:
            print(f"Erreur avec {filepath} : {e}")


def create_parser():
    """
    Cree le parser argparse
    """
    parser = argparse.ArgumentParser(
        description='Mini-grep : recherche de motifs dans des fichiers',
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog="""
Exemples :
  mini_grep.py "error" file.log
  mini_grep.py -i "warning" *.log
  mini_grep.py -rn "^ERROR.*database" app.log
  mini_grep.py -c "import" *.py
        """
    )
    
    parser.add_argument(
        'pattern',
        help='Motif a rechercher'
    )
    
    parser.add_argument(
        'files',
        nargs='+',
        help='Fichier(s) a analyser'
    )
    
    parser.add_argument(
        '-i', '--ignore-case',
        action='store_true',
        help='Ignorer la casse'
    )
    
    parser.add_argument(
        '-n', '--line-number',
        action='store_true',
        help='Afficher les numeros de ligne'
    )
    
    parser.add_argument(
        '-c', '--count',
        action='store_true',
        help='Afficher seulement le nombre de matches'
    )
    
    parser.add_argument(
        '-r', '--regex',
        action='store_true',
        help='Traiter le pattern comme une regex'
    )
    
    return parser


# Test de la fonction
print("Exemple d'utilisation de mini_grep :\n")

# Creer un fichier de test
test_file = Path('test_grep.txt')
test_file.write_text("""
This is line 1
ERROR: Something went wrong
This is line 3
WARNING: Check this
error in lowercase
Another ERROR here
Final line
""")

print("1. Recherche simple :")
mini_grep('ERROR', ['test_grep.txt'])

print("\n2. Insensible a la casse avec numeros de ligne :")
mini_grep('error', ['test_grep.txt'], ignore_case=True, line_number=True)

print("\n3. Compte uniquement :")
mini_grep('error', ['test_grep.txt'], ignore_case=True, count=True)

print("\n4. Avec regex :")
mini_grep(r'^ERROR.*wrong$', ['test_grep.txt'], use_regex=True, line_number=True)

# Nettoyer
test_file.unlink()

print("\n--- Aide du parser ---\n")
parser = create_parser()
parser.print_help()

---
## Section 4 : Dataclasses et Enum

### Exercice 4.1 - Configuration avec Dataclass (MOYEN)

Creez une dataclass `AppConfig` pour la configuration d'une application :

**Champs :**
- `app_name` : str
- `debug` : bool = False
- `database_url` : str
- `max_connections` : int = 10
- `timeout` : float = 30.0
- `allowed_hosts` : list[str] = field(default_factory=list)

**Methodes :**
- `validate()` : valide la configuration
- `from_dict(data)` : cree depuis un dict (classmethod)
- `to_dict()` : convertit en dict
- `from_env()` : charge depuis variables d'environnement (classmethod)

In [ ]:
# Votre code ici


### Solution 4.1

In [ ]:
# Solution
from dataclasses import dataclass, field, asdict
from typing import List
import os

@dataclass
class AppConfig:
    """Configuration d'application avec validation"""
    
    app_name: str
    database_url: str
    debug: bool = False
    max_connections: int = 10
    timeout: float = 30.0
    allowed_hosts: List[str] = field(default_factory=list)
    
    def __post_init__(self):
        """Validation apres initialisation"""
        self.validate()
    
    def validate(self):
        """Valide la configuration"""
        errors = []
        
        if not self.app_name:
            errors.append("app_name ne peut pas etre vide")
        
        if not self.database_url:
            errors.append("database_url ne peut pas etre vide")
        
        if self.max_connections <= 0:
            errors.append("max_connections doit etre > 0")
        
        if self.timeout <= 0:
            errors.append("timeout doit etre > 0")
        
        if errors:
            raise ValueError(f"Configuration invalide : {'; '.join(errors)}")
    
    @classmethod
    def from_dict(cls, data: dict):
        """Cree une configuration depuis un dictionnaire"""
        return cls(**data)
    
    def to_dict(self):
        """Convertit en dictionnaire"""
        return asdict(self)
    
    @classmethod
    def from_env(cls, prefix='APP_'):
        """Charge la configuration depuis les variables d'environnement"""
        # Convertir les noms de champs en variables d'environnement
        app_name = os.getenv(f'{prefix}NAME', 'MyApp')
        database_url = os.getenv(f'{prefix}DATABASE_URL', 'sqlite:///db.sqlite')
        debug = os.getenv(f'{prefix}DEBUG', 'False').lower() == 'true'
        max_connections = int(os.getenv(f'{prefix}MAX_CONNECTIONS', '10'))
        timeout = float(os.getenv(f'{prefix}TIMEOUT', '30.0'))
        
        allowed_hosts_str = os.getenv(f'{prefix}ALLOWED_HOSTS', '')
        allowed_hosts = [h.strip() for h in allowed_hosts_str.split(',') if h.strip()]
        
        return cls(
            app_name=app_name,
            database_url=database_url,
            debug=debug,
            max_connections=max_connections,
            timeout=timeout,
            allowed_hosts=allowed_hosts
        )


# Tests
print("=== TEST DATACLASS CONFIG ===")

# 1. Creation directe
print("\n1. Creation directe :")
config1 = AppConfig(
    app_name="MyApp",
    database_url="postgresql://localhost/mydb",
    debug=True,
    max_connections=50,
    allowed_hosts=['localhost', '127.0.0.1']
)
print(config1)

# 2. Depuis dictionnaire
print("\n2. Depuis dictionnaire :")
config_dict = {
    'app_name': 'TestApp',
    'database_url': 'sqlite:///test.db',
    'debug': False,
    'timeout': 60.0
}
config2 = AppConfig.from_dict(config_dict)
print(config2)

# 3. Conversion en dict
print("\n3. Conversion en dict :")
print(config2.to_dict())

# 4. Validation
print("\n4. Test de validation (config invalide) :")
try:
    config_invalid = AppConfig(
        app_name="",
        database_url="postgresql://localhost/db",
        max_connections=-5
    )
except ValueError as e:
    print(f"Erreur attrapee : {e}")

# 5. Depuis environnement
print("\n5. Depuis variables d'environnement :")
os.environ['APP_NAME'] = 'EnvApp'
os.environ['APP_DATABASE_URL'] = 'mysql://localhost/envdb'
os.environ['APP_DEBUG'] = 'True'
os.environ['APP_ALLOWED_HOSTS'] = 'localhost, example.com, api.example.com'

config3 = AppConfig.from_env()
print(config3)
print(f"Hosts autorises : {config3.allowed_hosts}")

### Exercice 4.2 - Systeme de Statuts avec Enum (DIFFICILE)

Creez un systeme de gestion de statuts de commandes :

1. **Enum `StatutCommande`** avec :
   - PENDING, CONFIRMED, PROCESSING, SHIPPED, DELIVERED, CANCELLED
   - Chaque valeur a un attribut `description`

2. **Dataclass `Commande`** avec :
   - id, statut, date_creation, date_modification
   - Methode `changer_statut(nouveau_statut)` avec validation des transitions

3. **Transitions valides** :
   - PENDING → CONFIRMED, CANCELLED
   - CONFIRMED → PROCESSING, CANCELLED
   - PROCESSING → SHIPPED, CANCELLED
   - SHIPPED → DELIVERED
   - (CANCELLED et DELIVERED sont finaux)

In [ ]:
# Votre code ici


### Solution 4.2

In [ ]:
# Solution
from enum import Enum
from dataclasses import dataclass, field
from datetime import datetime

class StatutCommande(Enum):
    """Statuts possibles d'une commande"""
    
    PENDING = ("En attente", False)
    CONFIRMED = ("Confirmee", False)
    PROCESSING = ("En preparation", False)
    SHIPPED = ("Expediee", False)
    DELIVERED = ("Livree", True)
    CANCELLED = ("Annulee", True)
    
    def __init__(self, description, est_final):
        self.description = description
        self.est_final = est_final
    
    def __str__(self):
        return self.description


# Matrice des transitions valides
TRANSITIONS_VALIDES = {
    StatutCommande.PENDING: [StatutCommande.CONFIRMED, StatutCommande.CANCELLED],
    StatutCommande.CONFIRMED: [StatutCommande.PROCESSING, StatutCommande.CANCELLED],
    StatutCommande.PROCESSING: [StatutCommande.SHIPPED, StatutCommande.CANCELLED],
    StatutCommande.SHIPPED: [StatutCommande.DELIVERED],
    StatutCommande.DELIVERED: [],  # Statut final
    StatutCommande.CANCELLED: [],  # Statut final
}


class TransitionInvalideError(Exception):
    """Erreur lors d'une transition de statut invalide"""
    pass


@dataclass
class Commande:
    """Commande avec gestion des statuts"""
    
    id: str
    statut: StatutCommande = StatutCommande.PENDING
    date_creation: datetime = field(default_factory=datetime.now)
    date_modification: datetime = field(default_factory=datetime.now)
    historique: list = field(default_factory=list, repr=False)
    
    def __post_init__(self):
        # Enregistrer le statut initial dans l'historique
        self._ajouter_historique(self.statut, "Creation")
    
    def _ajouter_historique(self, statut, raison=""):
        """Ajoute une entree dans l'historique"""
        self.historique.append({
            'statut': statut,
            'date': datetime.now(),
            'raison': raison
        })
    
    def changer_statut(self, nouveau_statut: StatutCommande, raison=""):
        """
        Change le statut de la commande avec validation
        """
        # Verifier si le statut actuel est final
        if self.statut.est_final:
            raise TransitionInvalideError(
                f"Impossible de modifier une commande avec le statut final '{self.statut}'"
            )
        
        # Verifier si la transition est valide
        transitions_autorisees = TRANSITIONS_VALIDES.get(self.statut, [])
        
        if nouveau_statut not in transitions_autorisees:
            transitions_str = ', '.join([str(s) for s in transitions_autorisees])
            raise TransitionInvalideError(
                f"Transition invalide : {self.statut} → {nouveau_statut}. "
                f"Transitions valides : {transitions_str}"
            )
        
        # Effectuer la transition
        ancien_statut = self.statut
        self.statut = nouveau_statut
        self.date_modification = datetime.now()
        self._ajouter_historique(nouveau_statut, raison)
        
        return f"Transition reussie : {ancien_statut} → {nouveau_statut}"
    
    def afficher_historique(self):
        """Affiche l'historique des changements de statut"""
        print(f"\nHistorique de la commande {self.id} :")
        print("=" * 60)
        for entry in self.historique:
            date_str = entry['date'].strftime('%Y-%m-%d %H:%M:%S')
            raison = f" ({entry['raison']})" if entry['raison'] else ""
            print(f"{date_str} - {entry['statut']}{raison}")


# Tests
print("=== SYSTEME DE GESTION DE STATUTS ===\n")

# Creer une commande
cmd = Commande(id="CMD-001")
print(f"Commande creee : {cmd.id}")
print(f"Statut initial : {cmd.statut}\n")

# Transitions valides
print("1. Confirmer la commande")
print(cmd.changer_statut(StatutCommande.CONFIRMED, "Paiement recu"))

print("\n2. Demarrer la preparation")
print(cmd.changer_statut(StatutCommande.PROCESSING, "En cours de preparation"))

print("\n3. Expedier")
print(cmd.changer_statut(StatutCommande.SHIPPED, "Colis confie au transporteur"))

print("\n4. Livrer")
print(cmd.changer_statut(StatutCommande.DELIVERED, "Livraison confirmee"))

# Afficher l'historique
cmd.afficher_historique()

# Tenter une transition invalide
print("\n5. Tentative de modification d'une commande livree (erreur attendue) :")
try:
    cmd.changer_statut(StatutCommande.CANCELLED)
except TransitionInvalideError as e:
    print(f"Erreur : {e}")

# Tester une annulation
print("\n--- Nouvelle commande avec annulation ---")
cmd2 = Commande(id="CMD-002")
cmd2.changer_statut(StatutCommande.CONFIRMED)
cmd2.changer_statut(StatutCommande.CANCELLED, "Demande du client")
cmd2.afficher_historique()

---
## Conclusion

Felicitations ! Vous maitrisez maintenant les outils de la bibliotheque standard Python.

### Points cles :
- Regex : parsing et validation de donnees
- Pathlib : manipulation moderne des fichiers et dossiers
- Argparse : creation de scripts CLI professionnels
- Dataclasses : structures de donnees avec validation
- Enum : gestion de statuts et constantes typees

### Prochaines etapes :
Passez au notebook **exercice-05-data-engineering.ipynb** pour un projet complet de data engineering avec Pandas, Pydantic et DuckDB.